Resources:
- Darts Forecasting 🎯 Deep Learning & Global Models
 https://www.kaggle.com/code/ferdinandberr/darts-forecasting-deep-learning-global-models
- Pytorch Forecasting => TemporalFusionTransformer
 https://www.kaggle.com/code/luisblanche/pytorch-forecasting-temporalfusiontransformer
- 🛒 Sales Forecasting | Using gluonts
 https://www.kaggle.com/code/antoinegruson/sales-forecasting-using-gluonts
- Store Sales - Time Series Forecasting
 https://www.kaggle.com/code/sdeogade/store-sales-time-series-forecasting